In [1]:
import sys
from pathlib import Path
import shutil
import json
import csv
import re
import math

from shapely.geometry import Point
import requests

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

%load_ext autoreload
%autoreload 2

TMP = DIR/'tmp'


In [2]:
# # Visually check that SRTM tiles cover NZ by 
# # making polygons and viewing them at https://geojson.io.

# tids = wt.SRTM_NZ_TILE_IDS
# collection = {
#     'type': 'FeatureCollection', 
#     'features': list(map(wt.build_feature, tids))
#     }
# json.dumps(collection)

In [3]:
# Initialize

wt.rm_paths(TMP)
#transmitters_path = DIR/'tests'/'data'/'transmitters.csv'
transmitters_path = DIR/'tests'/'data'/'transmitters_single.csv'
api_key = wt.get_secret('GITLAB_API_KEY') # Need this to download topo
high_definition = False
if high_definition:
    topography_path = TMP/'srtm1'
else:
    topography_path = TMP/'srtm3'


In [4]:
# Process transmitters

out_path = TMP/'splat_files'   
wt.process_transmitters(transmitters_path, out_path)

% ls {out_path}

GrayWireless_4.az  GrayWireless_4.el  GrayWireless_4.lrp  GrayWireless_4.qth


In [8]:
# Download topography

transmitters = wt.read_transmitters(transmitters_path)
print('transmitters=', transmitters)

tids = wt.get_covering_tiles_ids(transmitters)
print('Tiles to download =', tids)

wt.download_topography(tids, topography_path, 
  high_definition=high_definition, api_key=api_key)

% ls {topography_path}

transmitters= [{'antenna_downtilt': '', 'antenna_height': 10.0, 'frequency': 5725.0, 'network_name': 'Gray Wireless', 'extra': '', 'site_name': '4', 'power_eirp': 4.0, 'name': 'GrayWireless_4', 'polarization': 0.0, 'bearing': '0', 'latitude': -35.712247, 'horizontal_beamwidth': '', 'vertical_beamwidth': '', 'longitude': 174.336547}]
Tiles to download = ['S36E173', 'S36E174', 'S37E173', 'S37E174']
S36E173.SRTMGL3.hgt.zip  S37E173.SRTMGL3.hgt.zip
S36E174.SRTMGL3.hgt.zip  S37E174.SRTMGL3.hgt.zip


In [9]:
# Process topography

out_path = TMP/'splat_files'
wt.process_topography(topography_path, out_path, 
  high_definition=high_definition)

% ls {out_path}

-36:-35:185:186.sdf  -37:-36:185:186.sdf  GrayWireless_4.az  GrayWireless_4.lrp
-36:-35:186:187.sdf  -37:-36:186:187.sdf  GrayWireless_4.el  GrayWireless_4.qth


In [10]:
# Compute coverage

in_path = TMP/'splat_files'
out_path = TMP/'coverage_reports'
wt.compute_coverage(in_path, out_path, high_definition=high_definition)

% ls {out_path}

GrayWireless_4-ck.png  GrayWireless_4.png              GrayWireless_4.tif
GrayWireless_4.kml     GrayWireless_4-site_report.txt


In [11]:
# Compute satellite line-of-sight for the Optus D2 geostationary satellite 
# at 152 degrees longitude

in_path = (TMP/'srtm3'/'S36E173.SRTMGL3.hgt.zip').resolve()
satellite_lon = 152
tile_id = wt.get_tile_id(in_path)
out_path = TMP/'{!s}_satellite_los.tif'.format(tile_id)
wt.compute_satellite_los(in_path, satellite_lon, out_path)

In [ ]:
# Clean up

wt.rm_paths(TMP)